In [1]:
# Import data
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
import json

In [2]:
# Load data
data = pd.read_csv("../utils/history.csv")  # Update filename here

In [3]:
def recommend_products_for_latest(UID, user_item_matrix, U, sigma, Vt, top_n=16):
    latest_order_id = user_item_matrix.index.max()  # Get the highest order ID
    i = user_item_matrix.index.get_loc(latest_order_id)  # Get the index of the latest order
    user_row = U[i]

    # Predict ratings for all items
    predicted_ratings = np.dot(user_row, Vt)

    # Get product IDs with the highest predicted ratings
    top_n_indices = np.argsort(predicted_ratings)[::-1][:top_n]
    recommended_product_ids = user_item_matrix.columns[top_n_indices].tolist()

    return {"user_id": UID, "products": recommended_product_ids}

In [4]:
# Create a user-item matrix (pivot table)
totalRecommendation = []

for UID in data['ID'].unique():
    personFilter = data['ID'].isin([UID])
    personalHistory = data[personFilter]

    user_item_matrix = personalHistory.pivot_table(
    index=['order_id'],  # Users as rows
    columns='product_id',  # Products as columns
    values='quantity',
    aggfunc='sum',
    fill_value=0   
)
    # Convert the DataFrame to a NumPy array
    user_item_matrix_array = user_item_matrix.to_numpy().astype(np.float64) # Convert to NumPy array

    # Perform SVD (adjust k based on your dataset and desired accuracy-complexity trade-off)
    U, sigma, Vt = svds(user_item_matrix_array, k=1)  # Choose appropriate k

    # Reconstruct a low-rank approximation of the user-item matrix (optional)
    user_item_approx = np.dot(U, np.dot(np.diag(sigma), Vt))
    recommended_products = recommend_products_for_latest(UID, user_item_matrix=user_item_matrix, U=U, sigma=sigma, Vt=Vt)
    # print(recommended_products) # if you wish, you can print the values
    totalRecommendation += [recommended_products]

totalRecommendation

[{'user_id': 'yGbezIdeDnPB6kSlVxFKQsO4tvD3',
  'products': [19, 11, 10, 4, 16, 17, 5, 15, 21, 7, 13, 23, 18, 12, 28, 29]},
 {'user_id': 'ljul8hizGLbsDwKUIjHIJYXCnFF3',
  'products': [21, 15, 16, 17, 7, 11, 18, 10, 26, 1, 23, 30, 13, 20, 27, 3]},
 {'user_id': '3jrrdvJWJkhtyEk8qGIJmJJk8M92',
  'products': [5, 22, 13, 21, 12, 23, 9, 17, 27, 30, 26, 25, 3, 1, 7, 6]}]

In [5]:
with open('recommendations.json', 'w') as f:
    json.dump(totalRecommendation, f, indent = 4)